In [1]:
from helpers_new_2 import *

In [2]:
### inputs 
section_lines_path = "D:/2_Analytics/6_plan_vs_actual/11_7_2025_data/11-07-2025_Planned_vs_Actual/Proposed Dump & Pit Area from client/CROSS SECTION from client.shp"

## mine boundary 
mine_boundary_path = "D:/2_Analytics/6_plan_vs_actual/11_7_2025_data/11-07-2025_Planned_vs_Actual/Mine Boundary/Mine Boundary.shp"

## Dtms
dtm_itr1_path = "D:/2_Analytics/6_plan_vs_actual/11_7_2025_data/11-07-2025_Planned_vs_Actual/dtms/DTM_ACW_2024.tif"
dtm_itr1_year_ip =  2024                              #input("Enter iteration (e.g., itr1, itr2) or press Enter for default (itr1): ")
dtm_itr1_year_ip = dtm_itr1_year_ip if dtm_itr1_year_ip else "itr1"

dtm_itr2_path = "D:/2_Analytics/6_plan_vs_actual/11_7_2025_data/11-07-2025_Planned_vs_Actual/dtms/ACW_DTM_15cm_2025.tif"
dtm_itr2_year_ip = 2025                              #input("Enter iteration (e.g., itr2) or press Enter for default (itr2): ")
dtm_itr2_year_ip = dtm_itr2_year_ip if dtm_itr2_year_ip else "itr2"




## boundary polygons
planned_and_done_excavation_path = "D:/2_Analytics/6_plan_vs_actual/11_7_2025_data/11-07-2025_Planned_vs_Actual/Shapefile_Outputs/Planned_and_Done_Excavation.shp"
unplanned_and_done_excavtion_path = "D:/2_Analytics/6_plan_vs_actual/11_7_2025_data/11-07-2025_Planned_vs_Actual/Shapefile_Outputs/Unplanned_and_Done_Excavation.shp"
planned_and_used_dump_path = None
unplanned_and_used_dump_path = "D:/2_Analytics/6_plan_vs_actual/11_7_2025_data/11-07-2025_Planned_vs_Actual/Shapefile_Outputs/Unplanned_and_Used_Dump_Area.shp"


## output folder path 
output_folder_path = "D:/2_Analytics/6_plan_vs_actual/11_7_2025_data/11-07-2025_Planned_vs_Actual/3d_output"

# Load all section lines
section_lines_gdf = gpd.read_file(section_lines_path)
print(f"Total section lines: {len(section_lines_gdf)}")

def process_lines(section_lines_gdf, start_line=1, end_line=None):
    
    # Set end_line to the total number of rows if not specified
    if end_line is None:
        end_line = len(section_lines_gdf)

    all_section_results = []  # Collect data for final CSV here

    # Loop through the specified range
    for i, (idx, line_gdf) in enumerate(
        section_lines_gdf.iloc[start_line - 1:end_line].iterrows(),
        start=start_line
    ):
        print(f"\nProcessing line {i}/{end_line}")
        

        # Convert single row to GeoDataFrame
        line_gdf = gpd.GeoDataFrame([line_gdf], crs=section_lines_gdf.crs)
        print(len(line_gdf), "features in line_gdf")
        if len(line_gdf) != 1:
            print(f"Warning: line_gdf contains {len(line_gdf)} features instead of 1.")

        # Extract section name
        section_value = None
        for attr_name in ['SECTION', 'section']:
            if attr_name in line_gdf.columns:
                section_value = str(line_gdf.iloc[0][attr_name])
                break
    
        # Sanitize and create subfolder
        if section_value:
            section_name = section_value.replace(" ", "_").replace("/", "_").replace("\\", "_")
            section_folder = os.path.join(output_folder_path, f"section_{section_name}")
        else:
            section_name = f"line_{i+1}"
            section_folder = os.path.join(output_folder_path, section_name)
        #print(f"Section folder: {section_folder}")
        os.makedirs(section_folder, exist_ok=True)

   
        # Clip the line with the mine boundary
        clipped_gdf = clip_single_line_by_boundary(boundary_path = mine_boundary_path, 
                                                line_gdf = line_gdf, 
                                                output_folder = section_folder)
        #print(f"Clipped line features: {len(clipped_gdf)}")


        # Intersect clipped line with excavation/dump polygons
        (lines_planned_and_done_excavation,
            lines_unplanned_and_done_excavation,
            lines_planned_and_used_dump,
            lines_unplanned_and_used_dump
        ), section_name =               intersect_line_with_polygons_return_separately(
                                        line_gdf=clipped_gdf,
                                        output_folder_path=section_folder,
                                        section_name=section_name,
                                        planned_and_done_excavation_path=planned_and_done_excavation_path,
                                        unplanned_and_done_excavation_path=unplanned_and_done_excavtion_path,
                                        planned_and_used_dump_path=planned_and_used_dump_path,
                                        unplanned_and_used_dump_path=unplanned_and_used_dump_path
                                        )       

        print(f"Section name: {section_name}")

        # Extract elevation data from each DTM
        planned_and_done_excavation = extract_segment_df(lines_planned_and_done_excavation, clipped_gdf, dtm_itr1_path, dtm_itr2_path)
        unplanned_and_done_excavation = extract_segment_df(lines_unplanned_and_done_excavation, clipped_gdf, dtm_itr1_path, dtm_itr2_path)
        planned_and_used_dump = extract_segment_df(lines_planned_and_used_dump, clipped_gdf, dtm_itr1_path, dtm_itr2_path)
        unplanned_and_used_dump = extract_segment_df(lines_unplanned_and_used_dump, clipped_gdf, dtm_itr1_path, dtm_itr2_path)

        segment_data_dict = {
            "planned_and_done_excavation": (planned_and_done_excavation, "green"),
            "unplanned_and_done_excavation": (unplanned_and_done_excavation, "red"),
            "planned_and_used_dump": (planned_and_used_dump, "green"),
            "unplanned_and_used_dump": (unplanned_and_used_dump, "red"),
        }

        # Get elevation profiles
        elevations_itr1 = get_elevation_points_from_lines(dtm_itr1_path, clipped_gdf, interval=0.01)
        elevations_itr2 = get_elevation_points_from_lines(dtm_itr2_path, clipped_gdf, interval=0.01)

        # Convert elevation to DataFrame
        df1 = pd.DataFrame(elevations_itr1, columns=["x", "y", "z"])
        df1["chainage"] = [0.01 * i for i in range(len(df1))]

        df2 = pd.DataFrame(elevations_itr2, columns=["x", "y", "z"])
        df2["chainage"] = [0.01 * i for i in range(len(df2))]

        # Plot and save
        plot_multiple_intersections_on_elevation(
            df1, dtm_itr1_year_ip, 
            df2, dtm_itr2_year_ip, 
            segment_data_dict,
            section_name=section_name,
            output_folder_path=section_folder
        )

        # Store result for this section
        all_section_results.append((section_name, segment_data_dict))



        t_1 = time.time()
        output_dxf_path = os.path.join(section_folder, f"{section_name}_elevation_profile.dxf")
        export_to_dxf_with_colors_updated(df1, df2, segment_data_dict, output_dxf_path, section_name)
        t_2 = time.time()
        print(f"DXF export time: {t_2 - t_1:.2f} seconds")

    return all_section_results
        

all_section_results = process_lines(section_lines_gdf, start_line=1, end_line=None)

print(all_section_results)

# Export one CSV file
export_all_section_deviation_summary_csv(all_section_results, output_folder_path)


Total section lines: 18

Processing line 1/18
1 features in line_gdf
planned_and_done_excavation: No intersection.
planned_and_used_dump: File path is None or does not exist.
unplanned_and_used_dump: No intersection.
Section name: D_D'
Plot saved to: D:/2_Analytics/6_plan_vs_actual/11_7_2025_data/11-07-2025_Planned_vs_Actual/3d_output\section_D_D'\section_D_D'_plot\section_D_D'_plot_plot.png
Plotted ITR 1 in 193.693 seconds
Placed ITR 1 label at position: 1558.0900000000001 199.09647

Plotted ITR 2 in 192.139 seconds
Placed ITR 2 label at position: 1558.0900000000001 194.09647

Placed ITR 2 label at position: 1558.0900000000001 199.0862

Label Passed: planned_and_done_excavation
Zone: Excavation Status: Planned
Label Passed: unplanned_and_done_excavation
Zone: Excavation Status: Unplanned
Label Passed: planned_and_used_dump
Zone: Dump Status: Planned
Label Passed: unplanned_and_used_dump
Zone: Dump Status: Unplanned
DXF saved at: D:/2_Analytics/6_plan_vs_actual/11_7_2025_data/11-07-202